In [1]:
#Inspired by https://jnolis.com/blog/banned_license_plates/
#Following tutorial here: https://www.tensorflow.org/text/tutorials/text_generation

#load libraries

import tensorflow as tf

import numpy as np
import os
import time

In [2]:
#Get text of Ulysses from Project Gutenberg

path_to_file = tf.keras.utils.get_file('ulysses.txt', 'https://www.gutenberg.org/files/4300/4300-0.txt')


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1572272 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿The Project Gutenberg eBook of Ulysses, by James Joyce

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or r


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

126 unique characters


In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [8]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [9]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1572272,), dtype=int64, numpy=array([126,  51,  68, ...,   1,   2,   1])>

In [10]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [11]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

﻿
T
h
e
 
P
r
o
j
e


In [12]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [13]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [14]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [15]:
dataset = sequences.map(split_input_target)

In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [19]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 127) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  32512     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  130175    
                                                                 
Total params: 4,100,991
Trainable params: 4,100,991
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
EPOCHS = 30

In [26]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
243/243 [==============================] - 460s 2s/step - loss: 2.6512
Epoch 2/30
243/243 [==============================] - 441s 2s/step - loss: 2.0296
Epoch 3/30
243/243 [==============================] - 436s 2s/step - loss: 1.8024
Epoch 4/30
243/243 [==============================] - 456s 2s/step - loss: 1.6637
Epoch 5/30
243/243 [==============================] - 440s 2s/step - loss: 1.5724
Epoch 6/30
243/243 [==============================] - 437s 2s/step - loss: 1.5045
Epoch 7/30
243/243 [==============================] - 454s 2s/step - loss: 1.4458
Epoch 8/30
243/243 [==============================] - 444s 2s/step - loss: 1.3941
Epoch 9/30
243/243 [==============================] - 432s 2s/step - loss: 1.3444
Epoch 10/30
243/243 [==============================] - 427s 2s/step - loss: 1.2950
Epoch 11/30
243/243 [==============================] - 427s 2s/step - loss: 1.2457
Epoch 12/30
243/243 [==============================] - 500s 2s/step - loss: 1.1946
Epoch 13/30
2

In [27]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [28]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [36]:
start = time.time()
states = None
next_char = tf.constant(['Stephen'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Stephen, he had murdered to.

Alberrative parturine of eggliness had been knelt again with uncleaning pacts. She poured
before slop, slips for words. Softly. Somewhere. He tops so go suddenly _voglio_. Damn your
strength was more since Lord the day before neither for he was blue
fellow, not ready to set him to pink year and saw it with means which he halted
arphen sad Bloom whose requirements and personal congresslies that he was always for a
work of fear not to hurt you see and saw at the king’s name was a country bad
got off she went bling to how him? He’ll be all right she could wrinkled with her
cashancy with creamine bundle burying, passed out.

—Good Kinch and Joke’s head is, Mr Best said.

The bowl mether in _Abbes_’s and Varelagend Bloom with his hat.
Had he then he put the good in her necklike care and walking over it
will half know by the wailing soupens or something to come back with
drawn quiet thing like that. And Child’s daughter It lies in
Gibraltar who thereun 

_______

In [30]:
#save the trained model

tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets
